# PyDTA Samples


In this notebook, you can find 4 sample use cases of the PyDTA library.
1. Chemboost training
2. Normal training (w/o debiasing) for LMDTA model.
3. Debiased training for BPEDTA model with the guiding of BoWDTA.
4. Debiased training with a custom-defined model.

## Installation
We recommend you to use a virtual environment, since pip installer will install all libraries that are required to run this library, including torch and tensorflow. 

In [ ]:
!pip3 install pydta

# Load Sample Data

In [ ]:
from pydta.utils import load_sample_dta_data
from pydta.evaluation import evaluate_predictions

train_chemicals, train_proteins, train_labels = load_sample_dta_data(mini=True)['train']

## Chemboost

In [ ]:
from pydta.models import ChemBoost

prot_to_sb_chemicals = None # ??
chemboost = ChemBoost(prot_to_sb_chemicals=prot_to_sb_chemicals, n_estimators=1000)
chemboost.train(train_chemicals, train_proteins, train_labels)
preds = chemboost.predict(train_chemicals, train_proteins)
out = evaluate_predictions(train_labels, preds, ["r2"])
out

In [ ]:
del chemboost

## LMDTA (w/o Debiasing)

In [ ]:
from pydta.models import LMDTA

lmdta = LMDTA(n_epochs=1)
lmdta.train(train_chemicals, train_proteins, train_labels)
preds = lmdta.predict(train_chemicals, train_proteins)
out = evaluate_predictions(train_labels, preds, ["r2"])
out

In [ ]:
del lmdta

## BPEDTA (Debiased training with BoWDTA)

In [ ]:
from pydta.models import BoWDTA, BPEDTA, DebiasedDTA

debiaseddta = DebiasedDTA(BoWDTA, BPEDTA, predictor_params={'n_epochs': 1})
predictor = debiaseddta.train(train_chemicals, train_proteins, train_labels)
preds = predictor.predict(train_chemicals, train_proteins)
out = evaluate_predictions(train_labels, preds, ["r2"])
out

In [ ]:
del debiaseddta

## Custom Model Training with Debiasing 

In [ ]:
from pydta.models import IDDTA

class CustomDTAModel:
  # The constructor can have other arguments and/or the class have other attributes
  def __init__(self, n_epochs):
    self.n_epochs = n_epochs

  #The last argıment will be filled by DebiasedDTA
  def train(self, train_chemicals, train_proteins, train_labels, 
            sample_weights_by_epoch):
    pass

debiaseddta = DebiasedDTA(IDDTA, CustomDTAModel, predictor_params={'n_epochs': 1})
predictor = debiaseddta.train(train_chemicals, train_proteins, train_labels)
preds = predictor.predict(train_chemicals, train_proteins)
out = evaluate_predictions(train_labels, preds, ["r2"])
out